In [1]:
import time
import sqlite3
import os
import random
import numpy as np
import cupy as cp
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from PIL import Image
from scipy import interp
from itertools import cycle
from keras.models import Model, Input, load_model
from tensorflow.keras import datasets, layers, models
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, roc_curve, auc


#  Data Processing

In [ ]:
gear_defects = [0, 35, 76, 77]

In [2]:
# Image size
size = [300, 400]

In [3]:
# Load pre-processed dataset
X_train = np.load('gears_train_300x400_0,76,77,35.npy')
X_test = np.load('gears_test_300x400_0,76,77,35.npy')
y_train = np.load('gears_ytrain_300x400_0,76,77,35.npy')
y_test = np.load('gears_ytest_300x400_0,76,77,35.npy')

0.7327589988708496


# Training 

In [25]:
# Set up architecture
stride = 1
CHANNEL_AXIS = 1
dropout = 0.5
def res_layer(x ,filter1, filter2, pooling = False,dropout = 0.0):
    temp = x
    x = layers.Conv2D(filter1,(1,1),strides = stride,padding = "same")(x)
    x = layers.BatchNormalization(axis = CHANNEL_AXIS)(temp)
    x = layers.Activation("relu")(temp)
    
    x = layers.Conv2D(filter1,(3,3),strides = stride,padding = "same")(x)
    x = layers.BatchNormalization(axis = CHANNEL_AXIS)(temp)
    x = layers.Activation("relu")(temp)
    
    x = layers.Conv2D(filter2,(1,1),strides = stride,padding = "same")(x)
    x = layers.BatchNormalization(axis = CHANNEL_AXIS)(temp)
    x = layers.Activation("relu")(temp)
    
    x = layers.Add()([temp, x])
    if pooling:
        x = layers.MaxPooling2D((3,3), strides=2)(x)
    if dropout != 0.0:
        x = layers.Dropout(dropout)(x)
    x = layers.BatchNormalization(axis = CHANNEL_AXIS)(x)
    x = layers.Activation("relu")(x)
    return x

inp = Input(shape = X_train[0].shape)
x = inp
x = layers.Conv2D(64, (7, 7), padding='same')(x)
x = layers.BatchNormalization(axis = CHANNEL_AXIS)(x)
x = layers.Activation("relu")(x)
x = layers.MaxPooling2D((3, 3), strides=2)(x)
x = res_layer(x, 64, 256, dropout=dropout)
x = res_layer(x, 64, 256, dropout=dropout)
x = res_layer(x, 64, 256, dropout=dropout, pooling=True)
x = res_layer(x, 128, 512, dropout=dropout)
x = res_layer(x, 128, 512, dropout=dropout)
x = res_layer(x, 128, 512, dropout=dropout)
x = res_layer(x, 128, 512, dropout=dropout, pooling=True)
x = res_layer(x, 256, 1024, dropout=dropout)
x = res_layer(x, 256, 1024, dropout=dropout)
x = res_layer(x, 256, 1024, dropout=dropout)
x = res_layer(x, 256, 1024, dropout=dropout)
x = res_layer(x, 256, 1024, dropout=dropout)
x = res_layer(x, 256, 1024, dropout=dropout, pooling=True)
x = res_layer(x, 512, 2048, dropout=dropout)
x = res_layer(x, 512, 2048, dropout=dropout)
x = res_layer(x, 512, 2048, dropout=dropout, pooling=True)
x = layers.Flatten()(x)
x = layers.Dense(len(np.unique(y_train)), activation="softmax")(x)
resnet_model = Model(inp,x)
resnet_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
resnet_model.summary()


Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 300, 400, 1) 0                                            
__________________________________________________________________________________________________
conv2d_200 (Conv2D)             (None, 300, 400, 64) 3200        input_18[0][0]                   
__________________________________________________________________________________________________
batch_normalization_232 (BatchN (None, 300, 400, 64) 1200        conv2d_200[0][0]                 
__________________________________________________________________________________________________
activation_233 (Activation)     (None, 300, 400, 64) 0           batch_normalization_232[0][0]    
______________________________________________________________________________________

In [26]:
# Define early stopping and best model checkpoint
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, min_delta=0.1, patience=40)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_weights_only=True, mode='max', verbose=1, save_best_only=True)

In [27]:
# Fitting model with early stopping and checkpoint save
history = resnet_model.fit(X_train, y_train, epochs=50, 
                    validation_data=(X_test, y_test), callbacks=[es, mc])

Epoch 1/50
262/262 [==============================] - ETA: 0s - loss: 0.9685 - accuracy: 0.7781
Epoch 00001: val_accuracy improved from -inf to 0.75048, saving model to best_model.h5
262/262 [==============================] - 62s 238ms/step - loss: 0.9685 - accuracy: 0.7781 - val_loss: 1.3799 - val_accuracy: 0.7505
Epoch 2/50
262/262 [==============================] - ETA: 0s - loss: 0.9554 - accuracy: 0.7883
Epoch 00002: val_accuracy did not improve from 0.75048
262/262 [==============================] - 62s 236ms/step - loss: 0.9554 - accuracy: 0.7883 - val_loss: 1.3798 - val_accuracy: 0.7505
Epoch 3/50
262/262 [==============================] - ETA: 0s - loss: 0.9554 - accuracy: 0.7883
Epoch 00003: val_accuracy did not improve from 0.75048
262/262 [==============================] - 62s 236ms/step - loss: 0.9554 - accuracy: 0.7883 - val_loss: 1.3797 - val_accuracy: 0.7505
Epoch 4/50
262/262 [==============================] - ETA: 0s - loss: 0.9554 - accuracy: 0.7883
Epoch 00004: val_

Epoch 30/50
262/262 [==============================] - ETA: 0s - loss: 0.9554 - accuracy: 0.7883
Epoch 00030: val_accuracy did not improve from 0.75048
262/262 [==============================] - 61s 234ms/step - loss: 0.9554 - accuracy: 0.7883 - val_loss: 1.3793 - val_accuracy: 0.7505
Epoch 31/50
262/262 [==============================] - ETA: 0s - loss: 0.9554 - accuracy: 0.7883
Epoch 00031: val_accuracy did not improve from 0.75048
262/262 [==============================] - 61s 233ms/step - loss: 0.9554 - accuracy: 0.7883 - val_loss: 1.3793 - val_accuracy: 0.7505
Epoch 32/50
262/262 [==============================] - ETA: 0s - loss: 0.9554 - accuracy: 0.7883
Epoch 00032: val_accuracy did not improve from 0.75048
262/262 [==============================] - 61s 234ms/step - loss: 0.9554 - accuracy: 0.7883 - val_loss: 1.3793 - val_accuracy: 0.7505
Epoch 33/50
262/262 [==============================] - ETA: 0s - loss: 0.9554 - accuracy: 0.7883
Epoch 00033: val_accuracy did not improve fro

In [ ]:
# Load the best model
resnet_model.load_weights('best_model.h5')

In [28]:
resnet_model.save("resnet.h5")

In [29]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# save to json:  
hist_json_file = 'resnet_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# Metrics

In [ ]:
test_loss, test_acc = resnet_model.evaluate(X_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
# Show metrics
predictions = resnet_model.predict([X_test])
y_pred = predictions.argmax(axis=1).astype(int)
print(classification_report(y_test, y_pred))

In [ ]:
# Plot train and test validation of loss values and accuracy values
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'r--', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r--', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.ylim([0,1.5])
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# ROC binary label reshaping
y_pred_roc = (y_pred[:,None] == np.arange(y_pred.max()+1)).astype(int)
y_test_roc = (y_test[:,None] == np.arange(y_test.max()+1)).astype(int)

In [ ]:
# Plot ROC
def plot_roc(y_test, y_pred):
    # Plot linewidth.
    lw = 2
    n_classes = len(y_pred_roc[0])
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_roc[:, i], y_pred_roc[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_roc.ravel(), y_pred_roc.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure(1)
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()

plot_roc(y_test_roc,y_pred_roc)

In [ ]:
matrix = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(matrix, index = [i for i in range(len(np.unique(y_test)))],
                  columns = [i for i in range(len(np.unique(y_test)))])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, fmt='g')